In [21]:
import json

from torch.ao.nn.quantized.functional import threshold

# open and read content job_skills
with open("job_skill.json", "r", encoding='utf-8') as f:
    job_skills = json.load(f)

job_skills = [job for job in job_skills.values()]
import pandas as pd
import json

# open and read content job_skills
with open("job_skill.json", "r", encoding='utf-8') as f:
    job_skills = json.load(f)

all_job_skills = [skill["skill"] for profession in job_skills.values() for skill in profession]
unique_job_skills = list(set(all_job_skills))

print(unique_job_skills)
print(len(all_job_skills), len(unique_job_skills))

['kotlin', 'sap', 'azure functions', 'python', 'github', 'nosql', 'airflow', 'ci cd', 'vba', 'unreal engine', 'livedata', 'animation basics', 'confluence', 'cloud security', 'ssrs', 'navigation component', 'java ee', 'prototyping', '1с документооборот', 'data modeling', 'cassandra', 'sas', 'blueprints', 'user research', 'yii', 'r', 'databricks', 'golang', 'rxswift', 'azure data factory', 'git', 'helpdesk', 'ssis', 'viper', 'kubernetes', 'penetration testing', 'linq', 'ab testing', 'puppet', 'data cleaning', 'elk stack', 'multithreading', 'quantitative research', 'azure blob', 'webpack', 'cybersecurity', 'rxjava', 'tensorflow', 'postman', 'persona creation', 'windows', 'feature engineering', 'graphics programming', 'scrum', 'keras', 'angular', 'presentation', 'test case', '1c предприятие', 'branding basics', 'message queues', 'sql', 'laravel', 'express.js', 'data pipelines', 'network configuration', 'elasticsearch', 'power bi', 'message brokers', 'data visualization', 'etl tools', 'scss

In [22]:
import pandas as pd
import ast

df = pd.read_csv("courses_final.csv")

df['skills'] = df['skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

all_courses_skills = df['skills'].explode()
unique_course_skills = all_courses_skills.dropna().unique().tolist()

print(unique_course_skills)
print(len(all_courses_skills), len(unique_course_skills))


['NumPy', 'Data Manipulation', 'File Management', 'Data Structures', 'Application Programming Interface (API)', 'Data Analysis', 'Programming Principles', 'Web Scraping', 'Restful API', 'Object Oriented Programming (OOP)', 'Computer Programming', 'Jupyter', 'Python Programming', 'Data Import/Export', 'Pandas (Python Package)', 'Development Environment', 'Computational Thinking', 'Integrated Development Environments', 'Debugging', 'Algorithms', 'Problem Management', 'Network Protocols', 'Database Design', 'SQL', 'Data Collection', 'Data Visualization Software', 'Data Processing', 'Data Modeling', 'Test Driven Development (TDD)', 'Software Testing', 'Unit Testing', 'Plotly', 'Docker (Software)', 'Scripting', 'Git (Version Control System)', 'Data Ethics', 'Agile Methodology', 'Flask (Web Framework)', 'Web Development', 'Matplotlib', 'Statistical Modeling', 'Supervised Learning', 'Predictive Modeling', 'Regression Analysis', 'Data Cleansing', 'Data Pipelines', 'Descriptive Statistics', 'Fe

In [23]:
import re

def clean_skill(skill: str) -> str:
    cleaned = re.sub(r'\s*\(.*?\)', '', skill)
    return cleaned.strip().lower()

cleaned_course_skills = list(set([clean_skill(skill) for skill in unique_course_skills]))
print(cleaned_course_skills)

['sales', 'divide and conquer', 'embedded software', 'semantic web', 'data entry', 'language translation', 'nodes', 'user story', 'technical analysis', 'data as a service', 'network topology', 'statistical programming', 'prediction', '3d computer graphics', 'pesticides', 'securities trading', 'data storage', 'prototyping', 'blood pressure', 'linear discriminant analysis', 'full-stack web development', 'data processing', 'programming concepts', 'event-driven programming', 'localization', 'database permissions', 'user research', 'extensible markup language', 'statistical hypothesis testing', 'peer review', 'cron', 'computer-aided design', 'voice assistant technology', 'data analysis', 'language model', 'online service provider', 'usability', 'version control', 'analysis', 'software design', 'tensorflow', 'test automation', 'c', 'ecmascript 2015', 'customer service', 'microsoft 365', 'computer security', 'human readable', 'persuasive communication', 'applicant tracking systems', 'openshif

In [24]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/multilingual-e5-large')
# model = SentenceTransformer('multilingual-e5-large')

In [25]:
course_embeddings = model.encode(cleaned_course_skills, normalize_embeddings=True)
job_embeddings = model.encode(unique_job_skills, normalize_embeddings=True)


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(job_embeddings, course_embeddings)

In [29]:
import numpy as np

skill_map = {}
threshold = 0.85

for i, prof_skill in enumerate(unique_job_skills):
    top_indices = [
        idx for idx in np.argsort(similarity_matrix[i])[::-1]
            if threshold < similarity_matrix[i][idx] and not np.isclose(similarity_matrix[i][idx], 1.0)
    ][:3]
    if top_indices:
        skill_map[prof_skill] = [
            (cleaned_course_skills[idx], round(float(similarity_matrix[i][idx]), 2))
            for idx in top_indices
        ]

print(skill_map)
for prof_skill, matches in skill_map.items():
    print(f"Skill from job: {prof_skill}")
    for course_skill, score in matches:
        print(f"  --> {course_skill} (accuracy: {score})")
    print()

{'kotlin': [('matlab', 0.87), ('node.js', 0.86), ('compilers', 0.86)], 'azure functions': [('microsoft azure', 0.88), ('network functions virtualization', 0.87), ('callback functions', 0.86)], 'python': [('python programming', 0.91), ('syntax', 0.88), ('linux', 0.88)], 'github': [('stack overflow', 0.89), ('git', 0.89), ('version control', 0.88)], 'nosql': [('sql', 0.91), ('graphql', 0.89), ('mysql', 0.88)], 'airflow': [('dataflow', 0.9), ('apache airflow', 0.89), ('tensorflow', 0.86)], 'ci cd': [('ci/cd', 0.94), ('c', 0.86)], 'vba': [('matlab', 0.87), ('c#', 0.86), ('microsoft excel', 0.86)], 'unreal engine': [('unity engine', 0.91), ('3d engine', 0.9), ('virtual reality', 0.89)], 'livedata': [('digital data', 0.89), ('market data', 0.88), ('big data', 0.87)], 'animation basics': [('kinematics', 0.89), ('animations', 0.89), ('basic electrical systems', 0.88)], 'cloud security': [('cloud computing security', 0.95), ('cyber security', 0.94), ('cloud security infrastructure', 0.93)], 'na

In [33]:
from collections import defaultdict

reverse_skill_map = defaultdict(list)

for prof_skill, course_matches in skill_map.items():
    for course_skill, score in course_matches:
        reverse_skill_map[course_skill].append(prof_skill)

for course in reverse_skill_map:
    reverse_skill_map[course].sort(key=lambda x: x[0], reverse=True)

reverse_skill_map = dict(reverse_skill_map)

print(reverse_skill_map)

{'matlab': ['vba', 'matplotlib', 'kotlin', 'golang', 'gitlab'], 'node.js': ['vue.js', 'typescript', 'next.js', 'nginx', 'kotlin', 'jquery', 'javascript', 'express.js', 'es6', 'asp.net', 'angularjs', '.net core'], 'compilers': ['kotlin', 'blueprints'], 'microsoft azure': ['azure functions', 'azure data factory', 'azure blob', 'azure synapse', 'azure devops', 'azure', '.net core'], 'network functions virtualization': ['azure functions'], 'callback functions': ['azure functions'], 'python programming': ['python', 'graphics programming', '1с программирование'], 'syntax': ['python', 'markdown'], 'linux': ['unix', 'redis', 'python', 'android'], 'stack overflow': ['sharepoint', 'github'], 'git': ['junit', 'github', 'gitlab', 'go', 'bash'], 'version control': ['mvc', 'github'], 'sql': ['t sql', 'scss', 'spss', 'sql server', 'sqlalchemy', 'oracle', 'oracle pl sql', 'nosql', 'ms sql', 'mysql', 'linq'], 'graphql': ['rabbitmq', 'qlik', 'postgresql', 'oracle pl sql', 'nosql', 'grafana'], 'mysql': [

In [34]:
with open("reverse_skill_map.json", 'w', encoding='utf-8') as file:
    json.dump(reverse_skill_map, file, ensure_ascii=False, indent=4)

In [35]:
import ast

df = pd.read_csv("courses_final.csv")

def normalize_and_map(skill_list):
    normalized = []
    for skill in skill_list:
        cleaned = re.sub(r'\s*\(.*?\)', '', skill).strip().lower()
        mapped = reverse_skill_map.get(cleaned, cleaned)
        normalized.append(mapped if isinstance(mapped, str) else mapped[0])
    return normalized

df['skills'] = df['skills'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

df['mapped_skills'] = df['skills'].apply(normalize_and_map)

print("Пример замены:")
print(df[['skills', 'mapped_skills']].head(15))

df.to_csv('courses_normalized_skills.csv', index=False, encoding='utf-8-sig')

Пример замены:
                                               skills  \
0   [NumPy, Data Manipulation, File Management, Da...   
1   [Development Environment, Computational Thinki...   
2   [Network Protocols, Database Design, SQL, Appl...   
3   [Debugging, Test Driven Development (TDD), Int...   
4   [Plotly, Debugging, Test Driven Development (T...   
5   [Scripting, Data Analysis, Object Oriented Pro...   
6   [Data Import/Export, Matplotlib, Pandas (Pytho...   
7   [Development Environment, Programming Principl...   
8   [Unit Testing, Debugging, Image Analysis, Pyth...   
9   [Python Programming, Computer Programming, Dat...   
10  [Unit Testing, Debugging, Infrastructure as Co...   
11  [Scripting, Scripting Languages, Programming P...   
12  [Unsupervised Learning, Applied Machine Learni...   
13  [SQL, Query Languages, Jupyter, Databases, Dat...   
14  [Program Development, Application Development,...   

                                        mapped_skills  
0   [numpy, data